## Question 1:
Given the following table:
|Category|Subcategory|Sales (in $)|
|:--:|:--:|--:|
|Cate1|Subcate1|$30|
|Cate1|Subcate2|$40|
|Cate1|Subcate3|$25|
|Cate2|Subcate1|$60|
|Cate3|Subcate1|$40|
|Cate3|Subcate2|$75|
|Cate3|Subcate3|$45|
|Cate3|Subcate4|$80|
|Cate3|Subcate5|$80|

Get the highest sales in each category  
If tie happens for any subcategories then include those in the result as well

Expected Output:  
|Category|Highest_Sales_Subcategory|
|:--|:--|
|Cate1|Subcate2 :40|
|Cate2|Subcate1 :60|
|Cate3|Subcate4 :80, Subcate5 :80|

In [1]:
import psycopg2
con = psycopg2.connect(
    database='postgres',
    user='postgres',
    password='postgrepassword',
    host='localhost'
)
cur = con.cursor()

In [2]:
# Create table
table_name = 'interview_test_data'
cur.execute(f'select * from information_schema.tables where table_name=\'{table_name}\';')

if bool(cur.rowcount):
    print(f'Table {table_name} exist, dropping...')
    cur.execute(f'DROP TABLE {table_name};')

cur.execute(f'CREATE TABLE {table_name} (id serial PRIMARY KEY,  Category varchar(5), Subcategory varchar(8), Sales int);')

# Inserts
col1 = ['Cate1','Cate1','Cate1','Cate2','Cate3','Cate3','Cate3','Cate3','Cate3']
col2 = ['Subcate1','Subcate2','Subcate3','Subcate1','Subcate1','Subcate2','Subcate3','Subcate4','Subcate5',]
col3 = [30,40,25,60,40,75,45,80,80]
for i in range(len(col1)):
    cur.execute(f'INSERT INTO {table_name} (Category, Subcategory, Sales) VALUES (\'{col1[i]}\',\'{col2[i]}\',{col3[i]});')

# Check data
import pandas as pd
cur.execute(f'SELECT * FROM {table_name};')
table1 = pd.DataFrame(cur.fetchall(),columns=['Serial(PK)','Category','Subcategory','Sales'])
# print(table1)

# Failed attempt:
# query = """
# WITH sales_table AS (SELECT
# Category,
# Subcategory,
# Sales
# ROW_RANK() OVER(PARTITION BY Sales) AS rank
# FROM sales)
# SELECT
# Category,
# ARRAY_TO_STRING(ARRAY_AGG(CONCAT(Subcategory,” :”,sales)),”, “) AS highest_sales_subcategory
# FROM
# Sales_table
# WHERE rank = 1
# """

# Correction
query = """
WITH sales_table AS (SELECT
    Category,
    Subcategory,
    Sales,
    RANK() OVER(PARTITION BY Category ORDER BY Sales DESC) AS tbl_rank
FROM
    interview_test_data)
SELECT
    Category,
    ARRAY_TO_STRING(ARRAY_AGG(CONCAT(Subcategory,' :',Sales) ORDER BY CONCAT(Subcategory,' :',Sales) ASC),',') as pop
FROM
    sales_table
WHERE
    sales_table.tbl_rank = 1
GROUP BY
    Category
"""

cur.execute(query)
table1 = pd.DataFrame(cur.fetchall(),columns=['Category','Highest_Sales_Subcategory'])
cur.execute(f'DROP TABLE {table_name};')
table1
con.commit()


Table interview_test_data exist, dropping...


## Question 2: 

Table Category
|Category|Year|Sales|
|:--:|:--:|--:|
|Cate1|2015|$1,000|
|Cate2|2015|$4,000|
|Cate1|2017|$5,200|
|Cate1|2016|$3,400|
|Cate2|2016|$5,600|
|Cate1|2018|$5,800|
|Cate2|2017|$7,900|
|Cate2|2018|$2,400|

Comparision of Last vs current Year Sales  
Expected Output:
|Company|Year|Sales|Last Year Sales|
|:--:|:--:|:--:|--:|
|Cate1|2018|$5,800|5200|
|Cate1|2017|$5,200|3400|
|Cate1|2016|$3,400|1000|
|Cate1|2015|$1,000|NULL|
|Cate2|2018|$2,400|7900|
|Cate2|2017|$7,900|5600|
|Cate2|2016|$5,600|4000|
|Cate2|2015|$4,000|NULL|


In [3]:
# Create table
table_name = 'interview_test_data2'
cur.execute(f'select * from information_schema.tables where table_name=\'{table_name}\';')

if bool(cur.rowcount):
    print(f'Table {table_name} exist, dropping...')
    cur.execute(f'DROP TABLE {table_name};')

cur.execute(f'CREATE TABLE {table_name} (id serial PRIMARY KEY,  Category varchar(5), Year int, Sales int);')

# Inserts
col1 = ['Cate1','Cate2','Cate1','Cate1','Cate2','Cate1','Cate2','Cate2']
col2 = [2015,2015,2017,2016,2016,2018,2017,2018]
col3 = [1000,4000,5200,3400,5600,5800,7900,2400]

for i in range(len(col1)):
    cur.execute(f'INSERT INTO {table_name} (Category, Year, Sales) VALUES (\'{col1[i]}\',\'{col2[i]}\',{col3[i]});')

# Check data
import pandas as pd
cur.execute(f'SELECT * FROM {table_name};')
table1 = pd.DataFrame(cur.fetchall(),columns=['Serial(PK)','Category','Year','Sales'])
# print(table1)

# # Failed attempt: "Join with self, the key = key -1"

# Correction
query = """
SELECT
    a.Category AS Company,
    a.Year,
    a.Sales,
    b.Sales
FROM
    interview_test_data2 a
LEFT JOIN
    interview_test_data2 b ON a.Category = b.Category AND a.Year - 1 = b.Year
ORDER BY
    a.Category ASC, a.Year DESC
"""

cur.execute(query)
table2 = pd.DataFrame(cur.fetchall(),columns=['Company','Year','Sales','Sales Last Year'])
cur.execute(f'DROP TABLE {table_name};')
table2
con.commit()